# Magenta RT ile İnce Ayar (Fine-tuning)

**Hoş Geldiniz!** Bu Colab not defteri, Magenta RT müzik üretim modelini kendi ses verilerinizle ince ayar yapmanıza yardımcı olmak için tasarlanmıştır. İnce ayar, önceden eğitilmiş bir modeli alıp kendi özel veri kümeniz üzerinde ek eğitim yaparak modelin davranışını veri kümenizin özelliklerine (örneğin, belirli bir müzik stili) daha iyi uyacak şekilde adapte etme sürecidir.

**Bu Not Defteri Ne Yapar (Kavramsal Düzeyde):**
1.  **Kurulum:** Gerekli kütüphaneleri ve Magenta RT modelini yükler.
2.  **Veri Yükleme:** İnce ayar için kendi ses dosyalarınızı yüklemenize olanak tanır.
3.  **Veri Ön İşleme:** Yüklediğiniz ses dosyalarını modelin anlayabileceği bir formata (ses belirteçleri) dönüştürür.
4.  **Model ve Optimizatör Tanımlama (Kavramsal):** İnce ayar yapılacak model bileşenlerini ve eğitim için kullanılacak optimizasyon araçlarını tanımlar.
5.  **Eğitim Döngüsü (Kavramsal):** Modelin kendi verileriniz üzerinde eğitilmesini sağlar.
6.  **Modeli Kaydetme (Kavramsal):** İnce ayarlanmış modelinizi daha sonra kullanmak üzere kaydetmenize olanak tanır.

**ÖNEMLİ UYARILAR:**
*   **TPU Gereksinimi:** Bu not defteri, verimli bir şekilde çalışmak için bir **TPU çalışma zamanı** gerektirir (`Runtime > Change Runtime Type > TPU`).
*   **Deneysel Özellik:** Magenta RT için kullanıcı dostu ince ayar araçları aktif olarak geliştirilmektedir. Bu not defteri, sürecin **kavramsal bir uygulamasını** sunar. Özellikle Model Tanımlama (Adım 4) ve Eğitim Döngüsü (Adım 5) adımları, Magenta RT'nin altında yatan T5X modelinin karmaşık yapısı nedeniyle basitleştirilmiş veya yer tutucu kodlar içerebilir. Gerçek bir ince ayar, T5X'in kendi eğitim altyapısının daha derinlemesine entegrasyonunu gerektirebilir.
*   **Kaynak ve Zaman:** İnce ayar, veri kümenizin boyutuna ve seçtiğiniz eğitim parametrelerine bağlı olarak önemli miktarda zaman ve hesaplama kaynağı (özellikle TPU kullanımı) tüketebilir.
*   **Veri Hazırlığı:** Başlamadan önce, ince ayar için kullanacağınız ses dosyalarını (tercihen `.wav` formatında, hedef stile uygun, çeşitli ve kısa müzik parçaları) hazırlamanız önemlidir.

Lütfen her adımı dikkatlice okuyun ve talimatları izleyin. Başarılar!

## Adım 1: Kurulum ve Gerekli Bileşenlerin Başlatılması

Bu ilk adımda, Magenta RT kütüphanesini ve bağımlılıklarını yükleyip, ince ayar için temel alınacak olan önceden eğitilmiş Magenta RT modelini (ve ilişkili kodekleri) başlatacağız.

In [ ]:
# @title **Bu hücreyi çalıştırın** bağımlılıkları yüklemek için (~5 dakika)
# @markdown TPU çalışma zamanı kullandığınızdan emin olun (`Runtime > Change Runtime Type`).

# @markdown Colab, oturumu yeniden başlatmanızı isteyebilir. **Yeniden başlatmadan önce hücrenin çalışmasının bitmesini bekleyin!** Oturum yeniden başlatıldıktan sonra bir sonraki hücreye geçin.

print("Magenta RealTime deposu klonlanıyor...")
!git clone https://github.com/magenta/magenta-realtime.git

print("Gerekli TensorFlow sürümleri ayarlanıyor...")
# Magenta RT, TF nightly yapılarını gerektirir.
_all_tf = 'tensorflow tf-nightly tensorflow-cpu tf-nightly-cpu tensorflow-tpu tf-nightly-tpu tensorflow-hub tf-hub-nightly tensorflow-text tensorflow-text-nightly'
_nightly_tf = 'tf-nightly tensorflow-text-nightly tf-hub-nightly'

print("Kütüphane ve bağımlılıklar yükleniyor (TPU için)...")
!pip install -e magenta-realtime/[tpu] && pip uninstall -y {_all_tf} && pip install {_nightly_tf}

print("Kurulum tamamlandı. Eğer Colab tarafından istendiyse, şimdi 'Runtime > Restart Session' menüsünden oturumu yeniden başlatın ve bir sonraki hücreyle devam edin.")

In [ ]:
# @title **Oturumu yeniden başlattıysanız bu hücreyi çalıştırın** modeli ve diğer bileşenleri başlatmak için.
# @markdown Modelin ilk kez yüklenmesi (kontrol noktalarının indirilmesi) biraz zaman alabilir.

import os
import numpy as np
import tensorflow as tf # Tensorflow importu, TF nightly'nin düzgün yüklendiğini doğrulamak için de iyidir.

print(f"TensorFlow sürümü: {tf.__version__}")
print("Gerekli modüller yükleniyor...")
from magenta_rt import system
from magenta_rt import audio
from magenta_rt import musiccoca # MusicCoCa'nın da yüklenmesi gerekebilir, ancak ince ayarda doğrudan kullanılmayacak.
from magenta_rt import spectrostream
from magenta_rt.colab import utils # Demo Colab'daki rvq_to_llm vb. için
from google.colab import files
import librosa
import tarfile
import io # zipfile için gerekebilir
import zipfile # zip dosyalarını açmak için

print("Magenta RT modeli yükleniyor (lazy=True)...")
# Modeli lazy=True ile yüklüyoruz, böylece hemen belleğe alınmaz.
# İnce ayar için genellikle 'büyük' model tercih edilir, ancak kaynaklara göre 'base' de denenebilir.
MODEL_TAG = "large" #@param ["base", "large"]
MRT = None
codec = None # codec'i global yapmak için
try:
  MRT = system.MagentaRT(
      tag=MODEL_TAG, device="tpu:v2-8", skip_cache=True, lazy=True
  )
  print(f"Magenta RT '{MODEL_TAG}' modeli başarıyla yüklendi (veya yüklenecek şekilde ayarlandı).")
  # SpectroStream kodeğini alalım, veri ön işleme için gerekecek.
  codec = MRT.codec
  # Modelin kendisini (LLM) daha sonra `MRT._llm` ile alacağız, ancak bu nesnenin 
  # doğrudan eğitilebilir olup olmadığı veya T5X değişkenlerine nasıl erişileceği incelenmelidir.
  print("SpectroStream kodeği başarıyla alındı.")
except Exception as e:
  print(f"Model yüklenirken bir hata oluştu: {e}")
  print("Lütfen çalışma zamanının TPU olarak ayarlandığından ve önceki hücrenin başarıyla çalıştırıldığından emin olun.")

print("Başlatma ve hazırlık adımı tamamlandı.")

## Adım 2: İnce Ayar Verilerini Yükleme

Bu adımda, Magenta RT modelini ince ayar yapmak için kullanacağınız kendi ses dosyalarınızı yükleyeceksiniz.

**Desteklenen Formatlar:**
*   `.wav`
*   `.mp3`
*   `.zip` (içinde `.wav` veya `.mp3` dosyaları bulunan bir arşiv)

**Öneriler:**
*   **Kalite ve Çeşitlilik:** İnce ayar için yüksek kaliteli, çeşitli ve hedeflediğiniz stile uygun ses dosyaları kullanın. Model, verdiğiniz verinin özelliklerini öğrenmeye çalışacaktır.
*   **Dosya Uzunluğu:** Dosyalar çok uzun olmamalıdır (örneğin, her biri birkaç saniye ile bir dakika arasında). Daha uzun dosyalar bir sonraki adımda otomatik olarak daha kısa parçalara ayrılacaktır.
*   **Miktar:** Daha fazla veri genellikle daha iyi sonuçlar verir, ancak Colab'ın kaynak sınırlarını ve işlem süresini de göz önünde bulundurun.
*   **ZIP Kullanımı:** Çok sayıda dosya yükleyecekseniz, bunları tek bir ZIP arşivinde birleştirmek ve yüklemek daha pratiktir.

In [ ]:
# @title Veri Yükleme Hücresi

UPLOAD_METHOD = "Dosya Yükle" #@param ["Dosya Yükle", "Google Drive'dan Bağla"]
#@markdown Ses dosyalarınızın yükleneceği veya bulunacağı geçici dizin:
UPLOAD_PATH = "fine_tuning_data/uploaded/" #@param {type:"string"}
#@markdown --- 
#@markdown **"Dosya Yükle"** seçeneği için:
#@markdown Bu hücreyi çalıştırdıktan sonra "Dosya Seç" düğmesi görünecektir. Birden fazla dosya veya bir ZIP dosyası seçebilirsiniz.
#@markdown --- 
#@markdown **"Google Drive'dan Bağla"** seçeneği için:
#@markdown Google Drive'ınızdaki ses dosyalarını içeren klasörün yolunu belirtin (örneğin, `/MyDrive/my_music_data/`).
DRIVE_PATH = "/content/drive/MyDrive/fine_tuning_audio/" #@param {type:"string"}

if not os.path.exists(UPLOAD_PATH):
  os.makedirs(UPLOAD_PATH)
  print(f"'{UPLOAD_PATH}' dizini oluşturuldu.")

uploaded_file_paths = [] # Yüklenen veya Drive'dan bulunan dosyaların ilk listesi

if UPLOAD_METHOD == "Dosya Yükle":
  print(f"Lütfen ince ayar için ses dosyalarınızı (veya .zip dosyanızı) seçin.")
  try:
    # Birden fazla dosya yüklemeye izin vermek için files.upload() çağrısını döngü dışında tutun
    uploaded_files_dict = files.upload()
    if not uploaded_files_dict:
        print("Hiç dosya seçilmedi.")
    else:
        for filename, data in uploaded_files_dict.items():
          filepath = os.path.join(UPLOAD_PATH, filename)
          with open(filepath, 'wb') as f:
            f.write(data)
          print(f"'{filename}' dosyası '{filepath}' olarak kaydedildi.")
          uploaded_file_paths.append(filepath)
  except Exception as e:
    # 'NotFoundError' genellikle kullanıcı 'Cancel' düğmesine bastığında oluşur.
    if 'NotFoundError' in str(e) or 'No files were selected' in str(e):
      print("Dosya yükleme işlemi iptal edildi veya hiç dosya seçilmedi.")
    else:
      print(f"Dosya yüklenirken bir hata oluştu: {e}")
elif UPLOAD_METHOD == "Google Drive'dan Bağla":
  from google.colab import drive
  try:
    drive.mount('/content/drive')
    print(f"Google Drive başarıyla bağlandı.")
    actual_drive_path = DRIVE_PATH.strip()
    if os.path.exists(actual_drive_path):
      print(f"Google Drive'daki '{actual_drive_path}' içeriği taranıyor...")
      # Bu demoda, DRIVE_PATH'teki dosyaları doğrudan işleyeceğiz.
      # Bu nedenle UPLOAD_PATH'i DRIVE_PATH olarak ayarlıyoruz.
      UPLOAD_PATH = actual_drive_path 
      for dirname, _, filenames in os.walk(actual_drive_path):
        for filename in filenames:
          if filename.lower().endswith(('.wav', '.mp3', '.zip')):
            filepath = os.path.join(dirname, filename)
            uploaded_file_paths.append(filepath)
      if uploaded_file_paths:
        print(f"{len(uploaded_file_paths)} adet potansiyel dosya bulundu.")
      else:
        print(f"'{actual_drive_path}' içinde desteklenen formatta dosya bulunamadı (.wav, .mp3, .zip).")
    else:
      print(f"HATA: Google Drive'da belirtilen '{actual_drive_path}' yolu bulunamadı. Lütfen yolu kontrol edin.")
  except Exception as e:
    print(f"Google Drive bağlanırken bir hata oluştu: {e}")

if uploaded_file_paths:
  print("\nYüklenen ve/veya Google Drive'dan bulunan dosyaların listesi:")
  for p in uploaded_file_paths:
    print(f"  - {p}")
else:
  print("\nHiçbir dosya yüklenmedi veya belirtilen Google Drive yolunda bulunamadı.")

# ZIP dosyalarını açma (eğer varsa)
extracted_audio_files = []
if uploaded_file_paths:
  print("\nZIP dosyaları (varsa) ayıklanıyor...")
  # ZIP'ten çıkan dosyalar için ayrı bir alt dizin oluşturalım, karışmasın.
  # Eğer UPLOAD_PATH Google Drive ise, Drive'a yazmamak için yerel bir path kullanalım.
  zip_extract_base = "fine_tuning_data/extracted/" if UPLOAD_METHOD == "Dosya Yükle" else os.path.join(UPLOAD_PATH, "extracted_from_zip")
  if not os.path.exists(zip_extract_base):
      os.makedirs(zip_extract_base)

  processed_any_zip = False
  for filepath in uploaded_file_paths:
    if filepath.lower().endswith('.zip'):
      processed_any_zip = True
      print(f"  '{filepath}' ZIP dosyası işleniyor...")
      try:
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(zip_extract_base)
            print(f"    '{filepath}' içeriği '{zip_extract_base}' dizinine başarıyla ayıklandı.")
            # Ayıklanan dosyaları listeye ekle
            for member in zip_ref.namelist():
                if member.lower().endswith(('.wav', '.mp3')) and not member.startswith('__MACOSX'): # MAC OSX'in ürettiği gereksiz dosyaları atla
                    extracted_audio_files.append(os.path.join(zip_extract_base, member))
      except zipfile.BadZipFile:
        print(f"    HATA: '{filepath}' geçerli bir ZIP dosyası değil veya bozuk.")
      except Exception as e:
        print(f"    HATA: '{filepath}' ZIP dosyası ayıklanırken bir sorun oluştu: {e}")
    elif filepath.lower().endswith(('.wav', '.mp3')):
      # Doğrudan yüklenen (veya Drive'da bulunan) WAV/MP3 dosyalarını da listeye ekle
      extracted_audio_files.append(filepath)
  
  if not processed_any_zip:
      print("İşlenecek ZIP dosyası bulunamadı.")

# Tekrarları kaldır ve son listeyi oluştur
final_audio_files = sorted(list(set(extracted_audio_files)))
if final_audio_files:
  print("\nİnce ayar için kullanılacak son ses dosyaları:")
  for f_path in final_audio_files:
    print(f"  - {f_path}")
else:
  print("\nİnce ayar için işlenecek uygun formatta (.wav, .mp3) ses dosyası bulunamadı. Lütfen yüklemelerinizi kontrol edin.")


## Adım 3: Veri Ön İşleme

Bu adımda, bir önceki adımda belirlenen ses dosyaları ince ayar için uygun formata getirilecektir:
1.  **Yeniden Örnekleme ve Kanal Ayarlama:** Tüm ses dosyaları SpectroStream kodeğinin beklediği örnekleme hızına (48kHz) ve kanal sayısına (stereo) dönüştürülür.
2.  **Parçalara Bölme (Chunking):** Ses dosyaları, belirlenen uzunlukta daha kısa parçalara (chunk) bölünür. Bu, modelin sabit uzunlukta girdilerle çalışmasını sağlar.
3.  **Belirteçleme (Tokenization):** Her bir ses parçası, SpectroStream kodeği kullanılarak ses belirteçlerine (audio tokens) dönüştürülür. Bu belirteçler, LLM'in anlayabileceği ayrık temsillerdir.

In [ ]:
# @title Ses Dosyalarını Ön İşleme Hücresi

#@markdown ### Ön İşleme Parametreleri:
TARGET_SAMPLE_RATE = 48000  # SpectroStream'in beklediği örnekleme hızı (Hz)
TARGET_CHANNELS = 2       # SpectroStream'in beklediği kanal sayısı (1=mono, 2=stereo)
#@markdown --- 
#@markdown ### Parça Ayarları:
CHUNK_DURATION_SECONDS = 2.0 #@param {type:"number", min:0.5, max:10, step:0.1}
#@markdown Her bir ses parçasının saniye cinsinden uzunluğu. Modelin orijinal `chunk_length`'i (2.0s) ile aynı veya yakın olması genellikle iyi bir başlangıç noktasıdır.
HOP_DURATION_SECONDS = 2.0 #@param {type:"number", min:0.1, max:10, step:0.1}
#@markdown Parçalar arasındaki atlama süresi (saniye). 
#@markdown *   Eğer `HOP_DURATION_SECONDS == CHUNK_DURATION_SECONDS` ise, parçalar arasında overlap olmaz (bağımsız parçalar).
#@markdown *   Eğer `HOP_DURATION_SECONDS < CHUNK_DURATION_SECONDS` ise, parçalar arasında overlap olur (veri artırma tekniği olarak kullanılabilir).
#@markdown *   Overlap kullanmak, eğitim veri miktarını artırsa da, ince ayar stratejisini ve hedeflerin nasıl oluşturulacağını etkileyebilir.

processed_tokens_list = [] # İşlenmiş tüm parçaların belirteçlerini tutacak liste
failed_files_preprocessing = [] # Ön işleme sırasında hata veren dosyalar

if 'final_audio_files' not in locals() or not final_audio_files:
  print("HATA: İşlenecek ses dosyası listesi bulunamadı. Lütfen Adım 2'yi çalıştırarak ses dosyalarınızı yükleyin.")
else:
  if MRT is None or codec is None:
    print("HATA: Magenta RT modeli (MRT) veya SpectroStream kodeği (codec) yüklenemedi. Lütfen Adım 1'deki başlatma hücresini kontrol edin.")
  else:
    chunk_length_samples = int(CHUNK_DURATION_SECONDS * TARGET_SAMPLE_RATE)
    hop_length_samples = int(HOP_DURATION_SECONDS * TARGET_SAMPLE_RATE)
    min_samples_for_one_chunk = chunk_length_samples

    print(f"Toplam {len(final_audio_files)} adet ses dosyası ön işleme için sırada...")
    for i, audio_filepath in enumerate(final_audio_files):
      print(f"\n[{i+1}/{len(final_audio_files)}] İşleniyor: {audio_filepath}")
      try:
        # Ses dosyasını yükle (orijinal sr ve kanallarla)
        y, sr = librosa.load(audio_filepath, sr=None, mono=False) 
        current_channels = y.ndim if y.ndim == 1 else y.shape[0] if y.shape[0] < y.shape[1] else y.shape[1]
        current_samples = y.shape[0] if y.ndim == 1 else y.shape[1] if y.shape[0] < y.shape[1] else y.shape[0]
        print(f"  Orijinal: {sr} Hz, {current_samples} örnek, {current_channels} kanal")

        # Librosa bazen (kanal, örnek) şeklinde yükleyebilir, (örnek, kanal) formatına getir
        if y.ndim == 2 and y.shape[0] < y.shape[1] and y.shape[0] in [1,2]: # (kanal, örnek) ise transpoze et
            y = y.T
            print("  Format (örnek, kanal) olarak ayarlandı.")

        # Örnekleme hızını hedef hıza dönüştür
        if sr != TARGET_SAMPLE_RATE:
          y = librosa.resample(y, orig_sr=sr, target_sr=TARGET_SAMPLE_RATE, axis=0) # axis=0 zaman ekseni
          print(f"  {TARGET_SAMPLE_RATE} Hz'e yeniden örneklendi.")
        
        # Kanal sayısını hedef kanal sayısına (stereo) dönüştür
        if y.ndim == 1: # Mono ise stereo yap
          y = np.stack([y, y], axis=-1)
          print("  Mono'dan Stereo'ya dönüştürüldü (kopyalayarak).")
        elif y.shape[1] != TARGET_CHANNELS:
          print(f"  Uyarı: Beklenmedik kanal sayısı ({y.shape[1]}). Stereo'ya dönüştürme denenecek.")
          if y.shape[1] > TARGET_CHANNELS: # Fazla kanal varsa ilk ikisini al
            y = y[:, :TARGET_CHANNELS]
          else: # Tekrar mono durumu (çok kanallı ama TARGET_CHANNELS'dan az ise, bu durum nadirdir)
            y_mono = librosa.to_mono(y.T) # Önce mono yap
            y = np.stack([y_mono, y_mono], axis=-1) # Sonra stereo yap
          print(f"  Stereo'ya dönüştürüldü (son kanal sayısı: {y.shape[1]}).")
        
        num_samples = y.shape[0]
        if num_samples < min_samples_for_one_chunk:
          print(f"  UYARI: Dosya çok kısa ({num_samples} örnek), en az {min_samples_for_one_chunk} örnek ({CHUNK_DURATION_SECONDS}s) gerekli. Bu dosya atlanıyor.")
          failed_files_preprocessing.append(audio_filepath)
          continue
        
        print(f"  Son format (işleme için): {TARGET_SAMPLE_RATE} Hz, {y.shape[0]} örnek, {y.shape[1]} kanal")

        # Parçalara böl ve belirteçle
        file_tokens_count = 0
        for start_sample in range(0, num_samples - chunk_length_samples + 1, hop_length_samples):
          end_sample = start_sample + chunk_length_samples
          chunk_samples = y[start_sample:end_sample, :] # (chunk_length_samples, TARGET_CHANNELS)
          
          # magenta_rt.audio.Waveform nesnesi oluştur (float32 bekler)
          waveform_chunk = audio.Waveform(samples=chunk_samples.astype(np.float32), sample_rate=TARGET_SAMPLE_RATE)
          
          # SpectroStream ile belirteçle
          tokens = codec.encode(waveform_chunk) # Çıktı: (çerçeve_sayısı, tam_rvq_derinliği)
          
          # Eğitim için LLM'in beklediği RVQ derinliğine (decoder_codec_rvq_depth) indirgeyelim.
          # SpectroStream.encode() tam derinliği (örn. 64) döndürür.
          # LLM ise daha azını (örn. 16) kullanır.
          tokens_for_llm_training = tokens[:, :MRT.config.decoder_codec_rvq_depth]
          processed_tokens_list.append(tokens_for_llm_training)
          file_tokens_count += 1
        
        if file_tokens_count > 0:
            print(f"  Dosyadan {file_tokens_count} adet {CHUNK_DURATION_SECONDS}sn'lik parça işlendi ve '{MRT.config.decoder_codec_rvq_depth}' RVQ derinliğinde belirteçlendi.")
        else:
            print(f"  Dosyadan hiç tam parça çıkarılamadı (uzunluk: {num_samples} örnek, gereken: {chunk_length_samples} örnek). Atlanıyor.")
            failed_files_preprocessing.append(audio_filepath)

      except Exception as e:
        print(f"  HATA: '{audio_filepath}' işlenirken bir sorun oluştu: {e}")
        import traceback
        traceback.print_exc()
        failed_files_preprocessing.append(audio_filepath)
    
    print(f"\n--- Ön İşleme Tamamlandı ---")
    print(f"Toplam {len(processed_tokens_list)} adet ses parçası ({CHUNK_DURATION_SECONDS}sn her biri) başarıyla işlendi ve belirteçlendi.")
    if failed_files_preprocessing:
      print("Aşağıdaki dosyalar ön işleme sırasında hatalarla karşılaştı veya atlandı:")
      for f_err in list(set(failed_files_preprocessing)):
        print(f"  - {f_err}")

if processed_tokens_list:
  print(f"\nİlk işlenen parçanın belirteç boyutu: {processed_tokens_list[0].shape}")
  expected_frames = int(CHUNK_DURATION_SECONDS * codec.config.frame_rate)
  actual_frames = processed_tokens_list[0].shape[0]
  actual_depth = processed_tokens_list[0].shape[1]
  print(f"  Beklenen çerçeve sayısı (her parça için): ~{expected_frames}, Alınan: {actual_frames}")
  print(f"  Alınan (ve eğitime verilecek) RVQ derinliği: {actual_depth} (Bu, LLM'in `decoder_codec_rvq_depth` değeri olan {MRT.config.decoder_codec_rvq_depth} olmalıdır)")
  if actual_depth != MRT.config.decoder_codec_rvq_depth:
      print(f"  UYARI: Belirteçlerin RVQ derinliği ({actual_depth}) LLM'in beklediği derinlikten ({MRT.config.decoder_codec_rvq_depth}) farklı! Bu durum eğitimde sorunlara yol açabilir.")

## Adım 4: İnce Ayar Modeli ve Optimizatörün Tanımlanması (Kavramsal)

Bu adımda, ince ayar yapılacak modeli (Magenta RT'nin LLM kısmı) ve eğitim için kullanılacak optimizatörü tanımlayacağız.

**ÖNEMLİ UYARI:** Magenta RT'nin altında yatan T5X modelinin parametrelerine ve eğitim fonksiyonlarına (Flax `apply_fn`) doğrudan erişmek ve bunları standart bir JAX/Flax eğitim döngüsüne entegre etmek karmaşıktır. `MagentaRT` sınıfı, çıkarım (inference) için optimize edilmiştir ve eğitim için doğrudan bir arayüz sunmayabilir. 
Bu nedenle, bu bölümdeki kodlar **büyük ölçüde kavramsal ve yer tutucudur.** Gerçek bir ince ayar için:
1.  T5X kütüphanesinin kendi ince ayar ve eğitim komut dosyalarını kullanmak (bu Colab'ın kapsamı dışındadır).
2.  Magenta RT'nin LLM modelini T5X'ten çıkarıp, eğitilebilir bir Flax modeline dönüştürmek ve bu model için `TrainState` oluşturmak gerekebilir. Bu, model mimarisinin ve ağırlık yükleme mekanizmalarının derinlemesine anlaşılmasını gerektirir.

Aşağıdaki hücreler, eğer böyle bir erişim mümkün olsaydı nasıl bir yol izlenebileceğine dair bir fikir vermek amacıyla sunulmuştur.

In [ ]:
# @title Model, Optimizatör ve Kayıp Fonksiyonunu Tanımlama (Kavramsal Kod)
import jax
import jax.numpy as jnp
import optax # JAX için popüler bir optimizasyon kütüphanesi
from flax.training import train_state  # Eğitim durumunu yönetmek için
# from flax.core import freeze, unfreeze # Parametreleri dondurmak/çözmek için gerekebilir

#@markdown ### İnce Ayar Parametreleri (Kavramsal):
LEARNING_RATE = 1e-5 #@param {type:"number"}
#@markdown Düşük bir öğrenme hızı genellikle ince ayar için daha güvenlidir.
WARMUP_STEPS = 50 #@param {type:"integer"}
TRAIN_BATCH_SIZE = 2 #@param {type:"integer"}
#@markdown `TRAIN_BATCH_SIZE` TPU üzerinde çalışırken genellikle TPU çekirdek sayısının (örn. 8) katları veya bölenleri olarak ayarlanır.
#@markdown Ancak, bellek kısıtlamaları nedeniyle daha küçük tutulabilir.

model_params_for_fine_tuning = None
model_apply_fn_for_fine_tuning = None
tx = None # Optimizatör
current_train_state = None # Flax TrainState

if MRT is None:
  print("HATA: Magenta RT modeli (MRT) yüklenemedi. Lütfen Adım 1'i kontrol edin.")
else:
  print("UYARI: Bu bölümdeki model ve optimizatör tanımlaması, Magenta RT'nin iç yapısına\
        erişimin mümkün olduğu varsayımına dayanmaktadır ve büyük ölçüde spekülatiftir.")

  # Kavramsal Adım 1: MRT._llm içinden T5X modelinin parametrelerini (ağırlıklarını) çıkarmak.
  # Bu, MRT._llm nesnesinin yapısına bağlıdır. `MRT._llm._model.get_variables()` gibi bir şey olabilir.
  try:
    # `MRT._llm` bir callable. Bu callable'ın `model` veya `variables` gibi bir özelliği olabilir.
    # Bu, `magenta_rt.depthformer.model.InteractiveModel` sınıfının iç yapısına bağlıdır.
    # Eğer T5X `CheckpointContent` nesnesine benzer bir yapı varsa, oradan değişkenler alınabilir.
    # Bu sadece bir tahmin:
    if hasattr(MRT, '_llm') and hasattr(MRT._llm, '_model_variables'):
        model_params_for_fine_tuning = MRT._llm._model_variables
        # Parametrelerin eğitilebilir olması için 'unfreeze' gerekebilir, eğer Flax'ta 'frozen dict' ise.
        # model_params_for_fine_tuning = unfreeze(model_params_for_fine_tuning)
        print("Model parametreleri (kavramsal olarak) alındı.")
    else:
        print("UYARI: MRT._llm._model_variables bulunamadı. Parametreler alınamadı.")

    # Kavramsal Adım 2: Modelin `apply_fn`'sini (ileri yayılım fonksiyonu) almak.
    # Bu da `MRT._llm`'in veya içindeki T5X modelinin bir özelliği olmalıdır.
    # `MRT._llm._predict_fn` çıkarım için kullanılır, eğitim için farklı bir `apply_fn` gerekebilir.
    # Veya `MRT._llm.model.apply` gibi bir şey olabilir.
    if hasattr(MRT, '_llm') and hasattr(MRT._llm, '_predict_fn'): #veya ._model.apply
        # `_predict_fn` genellikle eğitim için uygun değildir (dropout vs. kapalı olabilir).
        # Gerçek bir T5X model nesnesinin `apply` metoduna ihtiyacımız var.
        # Bu Colab'da bu fonksiyonu elde etmek zor olduğundan, yer tutucu bırakıyoruz.
        model_apply_fn_for_fine_tuning = None # Yer tutucu
        print("Model apply_fn (kavramsal olarak) referans alındı (ancak eğitim için uygun olmayabilir).")
        if model_apply_fn_for_fine_tuning is None:
            print("UYARI: Eğitim için uygun bir model_apply_fn bulunamadı.")
  except Exception as e:
    print(f"Model parametrelerini/apply_fn'yi alırken hata: {e}")

  if model_params_for_fine_tuning is not None and model_apply_fn_for_fine_tuning is not None:
    print("Optimizatör ve Eğitim Durumu oluşturuluyor...")
    lr_schedule = optax.warmup_cosine_decay_schedule(
        init_value=0.0, peak_value=LEARNING_RATE, warmup_steps=WARMUP_STEPS,
        decay_steps=10000, end_value=LEARNING_RATE/10 # decay_steps toplam adım sayısına göre ayarlanmalı
    )
    tx = optax.adamw(learning_rate=lr_schedule)
    
    try:
      current_train_state = train_state.TrainState.create(
          apply_fn=model_apply_fn_for_fine_tuning, 
          params=model_params_for_fine_tuning, 
          tx=tx
      )
      print("Flax TrainState başarıyla oluşturuldu.")
    except Exception as e:
      print(f"Flax TrainState oluşturulurken hata: {e}")
      print("Bu genellikle `apply_fn`'nin veya `params`'ın beklenen formatta olmamasından kaynaklanır.")
      current_train_state = None # Başarısız olduysa sıfırla
  else:
    print("Model parametreleri veya apply_fn eksik olduğu için optimizatör ve eğitim durumu oluşturulamadı.")
    # Optimizatörü yine de tanımlayalım ki bir sonraki hücre tamamen çökmesin (ama TrainState olmayacak)
    lr_schedule = optax.warmup_cosine_decay_schedule(
        init_value=0.0, peak_value=LEARNING_RATE, warmup_steps=WARMUP_STEPS,
        decay_steps=10000, end_value=LEARNING_RATE/10
    )
    tx = optax.adamw(learning_rate=lr_schedule)
    print("Optimizatör (tx) tanımlandı, ancak TrainState oluşturulamadı.")

# Kayıp Fonksiyonu (Cross-entropy)
# Bu fonksiyon, modelin logitlerini ve gerçek hedef belirteçlerini alır.
def compute_loss_for_training(logits, targets, vocab_size_for_loss_fn):
  # `targets` (batch_size, seq_len)
  # `logits` (batch_size, seq_len, vocab_size)
  one_hot_targets = jax.nn.one_hot(targets, num_classes=vocab_size_for_loss_fn)
  # Genellikle kayıp hesaplanırken PAD tokenları maskelenir.
  # Bu örnekte basitlik adına maskeleme eklenmemiştir.
  loss = optax.softmax_cross_entropy(logits=logits, labels=one_hot_targets).mean()
  return loss

print("Kayıp fonksiyonu (compute_loss_for_training) tanımlandı.")

if current_train_state is None:
    print("\nUYARI: Eğitim durumu (current_train_state) düzgün bir şekilde başlatılamadı. Eğitim adımı çalışmayacaktır.")

## Adım 5: Eğitim Döngüsü (Kavramsal)

Bu adımda, hazırlanan veriler üzerinde modelin ince ayarını gerçekleştirecek eğitim döngüsünü tanımlayacağız.

**YİNE UYARI:** Bu bölüm, Adım 4'teki `current_train_state`'in (Flax `TrainState` içeren model parametreleri, `apply_fn` ve optimizatör durumu) başarıyla oluşturulduğu varsayımına dayanır. Eğer `current_train_state` `None` ise, bu hücredeki eğitim döngüsü **çalışmayacaktır** ve sadece bir şablon olarak görev görecektir.

In [ ]:
# @title Eğitim Döngüsü Hücresi (Kavramsal Kod)
import time

#@markdown ### Eğitim Parametreleri:
NUM_EPOCHS = 1 #@param {type:"integer", min:1, max:100}
REPORT_INTERVAL_STEPS = 10 #@param {type:"integer", min:1, max:1000}
#@markdown İnce ayar için genellikle az sayıda epoch yeterlidir.

def prepare_training_data_iterator(tokens_list_for_epoch, batch_size_for_iterator, prng_key_for_iterator):
    """Veri kümesini karıştırır ve batch'ler halinde yield eder."""
    num_samples_total = len(tokens_list_for_epoch)
    if num_samples_total == 0:
        yield None, None # Veri yoksa None döndür
        return

    # Her epoch için veriyi karıştır
    perm = jax.random.permutation(prng_key_for_iterator, num_samples_total)
    shuffled_tokens_list = [tokens_list_for_epoch[i] for i in perm]

    for i in range(0, num_samples_total, batch_size_for_iterator):
        batch_tokens_raw = shuffled_tokens_list[i:i + batch_size_for_iterator]
        if not batch_tokens_raw: continue

        # Bu fonksiyon, Adım 4'teki prepare_training_batch'e benzer şekilde batch'i LLM formatına getirmeli.
        # Yani, RVQ derinliğini ayarlama, LLM vocab'ına dönüştürme ve padding.
        # Şimdilik, processed_tokens_list'in zaten doğru RVQ derinliğinde (decoder_codec_rvq_depth)
        # ve LLM vocab'ına dönüştürülmeye hazır olduğunu varsayıyoruz (Adım 3'te yapıldı).
        
        batch_input_llm_tokens_for_iter = []
        # İnce ayarda, genellikle `decoder_input_tokens` ve `decoder_target_tokens` olur.
        # `decoder_input_tokens` genellikle `[BOS, t1, t2, ..., tn]`
        # `decoder_target_tokens` ise `[t1, t2, ..., tn, EOS]` şeklindedir.
        # Bizim `processed_tokens_list`'imizdeki her eleman bir parça (chunk) belirteci.
        # Bu belirteçleri LLM kelime dağarcığına çevirmemiz gerekiyor.

        max_seq_len_this_batch = 0
        if batch_tokens_raw:
            max_seq_len_this_batch = max(raw_tok.shape[0] * raw_tok.shape[1] for raw_tok in batch_tokens_raw)

        for raw_tokens_chunk in batch_tokens_raw: # raw_tokens_chunk: (frames, rvq_depth)
            # RVQ derinliği Adım 3'te zaten MRT.config.decoder_codec_rvq_depth'e ayarlandı.
            llm_tokens_flat = utils.rvq_to_llm(
                raw_tokens_chunk, # Zaten doğru derinlikte olmalı
                MRT.config.codec_rvq_codebook_size,
                MRT.config.vocab_codec_offset
            ).reshape(-1) # (seq_len * depth) -> (total_seq_len,)
            
            # Sabit dizi uzunluğuna padding (max_seq_len_this_batch'e göre)
            # PAD token ID'si MRT.config.vocab_pad_token (genellikle 0)
            pad_amount = max_seq_len_this_batch - llm_tokens_flat.shape[0]
            padded_llm_tokens = jnp.pad(llm_tokens_flat, (0, pad_amount), constant_values=MRT.config.vocab_pad_token)
            batch_input_llm_tokens_for_iter.append(padded_llm_tokens)

        if not batch_input_llm_tokens_for_iter:
            yield None, None
            continue

        final_batch_inputs = jnp.stack(batch_input_llm_tokens_for_iter)
        # Hedefler, girdilerin bir sonraki token'ı olacak şekilde ayarlanır (sağa kaydırılmış).
        # Ve son token PAD ile değiştirilir.
        final_batch_targets = jnp.pad(final_batch_inputs[:, 1:], ((0,0), (0,1)), constant_values=MRT.config.vocab_pad_token)
        
        yield final_batch_inputs, final_batch_targets

# Eğitim adımı fonksiyonu (JIT için derlenebilir)
@jax.jit
def perform_train_step(current_state_in_jit, batch_decoder_input_tokens, batch_decoder_target_tokens):
    """Tek bir eğitim adımını gerçekleştirir."""
    def loss_calculation_fn(params_in_loss_fn):
        # Bu `apply_fn` çağrısı, modelin gerçek arayüzüne göre ayarlanmalıdır.
        # `MRT._llm` veya ondan türetilen Flax modelinin `apply` metodu olmalı.
        # Girdiler: {'params': params}, decoder_input_tokens=batch_decoder_input_tokens, training=True
        # Çıktı: logits (batch_size, seq_len, vocab_size)
        
        # ** YER TUTUCU / KAVRAMSAL ÇAĞRI **
        # Bu, `current_state_in_jit.apply_fn`'nin doğru şekilde tanımlandığını varsayar.
        # Gerçek T5X modeli, sadece decoder tokenları değil, encoder çıktılarını da bekleyebilir.
        # Bu durumda, `batch_encoder_input_tokens` ve `enable_dropout` gibi ek argümanlar gerekebilir.
        # Bu örnekte, `apply_fn`'in sadece `decoder_input_tokens` ve `training` flag'i aldığını varsayıyoruz.
        logits = current_state_in_jit.apply_fn(
            {'params': params_in_loss_fn}, 
            decoder_input_tokens=batch_decoder_input_tokens, 
            # encoder_input_tokens=... (eğer gerekliyse, dummy veya gerçek sağlanmalı)
            training=True # Dropout vb. aktif olması için
        )
        
        # Gerçek kelime dağarcığı boyutunu logit'lerden al
        actual_vocab_size = logits.shape[-1]
        loss = compute_loss_for_training(logits, batch_decoder_target_tokens, actual_vocab_size)
        return loss, logits # Kaybı ve logitleri döndür (logitler metrikler için kullanılabilir)

    # Gradyanları hesapla
    grad_calculation_fn = jax.value_and_grad(loss_calculation_fn, has_aux=True)
    (loss_value, _), grads = grad_calculation_fn(current_state_in_jit.params)
    
    # Gradyanları uygula ve yeni state'i al
    new_state_after_apply = current_state_in_jit.apply_gradients(grads=grads)
    return new_state_after_apply, loss_value


if 'processed_tokens_list' not in locals() or not processed_tokens_list:
    print("HATA: İşlenmiş veri (processed_tokens_list) bulunamadı. Lütfen Adım 3'ü çalıştırın.")
elif 'current_train_state' not in locals() or current_train_state is None:
    print("HATA: Model eğitim durumu (current_train_state) başlatılamadı. Lütfen Adım 4'ü kontrol edin.")
    print("Bu genellikle, Magenta RT'nin LLM'inin parametrelerine ve/veya eğitim için uygun bir `apply_fn`'sine erişilemediği anlamına gelir.")
    print("Bu Colab'da tam bir ince ayar döngüsü çalıştırmak için bu gereklidir ve şu anda bu desteklenmiyor olabilir.")
else:
    print("Eğitim döngüsü başlıyor (Adım 4'teki UYARILARI dikkate alın)...")
    
    total_steps_taken = current_train_state.step # Eğer daha önce eğitildiyse adımı koru
    accumulated_loss_for_report = 0.0
    epoch_start_time = time.time()
    prng_key_epoch_shuffling = jax.random.PRNGKey(42) # Her epoch için veri karıştırma anahtarı

    try:
      for epoch in range(NUM_EPOCHS):
          print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
          prng_key_epoch_shuffling, current_epoch_shuffle_key = jax.random.split(prng_key_epoch_shuffling)
          
          batch_iterator = prepare_training_data_iterator(processed_tokens_list, TRAIN_BATCH_SIZE, current_epoch_shuffle_key)
          
          step_in_epoch = 0
          for batch_inputs, batch_targets in batch_iterator:
              if batch_inputs is None or batch_targets is None:
                  if step_in_epoch == 0: print("  Bu epoch için veri bulunamadı veya batch oluşturulamadı.")
                  break # Iterator bitti veya hata oluştu
              
              # Eğitim adımını JIT ile derlenmiş fonksiyonu kullanarak çalıştır
              current_train_state, loss_val = perform_train_step(current_train_state, batch_inputs, batch_targets)
              accumulated_loss_for_report += loss_val.item()
              total_steps_taken += 1
              step_in_epoch += 1

              if total_steps_taken % REPORT_INTERVAL_STEPS == 0 and total_steps_taken > 0:
                  avg_loss_since_last_report = accumulated_loss_for_report / (REPORT_INTERVAL_STEPS if step_in_epoch >= REPORT_INTERVAL_STEPS else step_in_epoch % REPORT_INTERVAL_STEPS)
                  current_epoch_time = time.time() - epoch_start_time
                  print(f"  Epoch: {epoch+1}, Adım: {total_steps_taken}, Ortalama Kayıp (son {REPORT_INTERVAL_STEPS} adım): {avg_loss_since_last_report:.4f}, Epoch Süresi: {current_epoch_time:.2f}s")
                  accumulated_loss_for_report = 0.0
          
          print(f"Epoch {epoch + 1} tamamlandı. Toplam {step_in_epoch} batch işlendi.")
          epoch_start_time = time.time() # Sonraki epoch için zamanı sıfırla

    except NotImplementedError as nie:
        print(f"EĞİTİM DURDURULDU: {nie}")
        print("Lütfen Adım 4'teki model ve optimizatör tanımlamalarını ve `perform_train_step` içindeki `apply_fn` çağrısını kontrol edin.")
    except Exception as e:
        print(f"Eğitim sırasında beklenmedik bir hata oluştu: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print(f"--- Eğitim Döngüsü Sonlandı ---")
        if total_steps_taken > (current_train_state.step if hasattr(current_train_state, 'step') and current_train_state.step is not None else 0) :
            print(f"Toplam {total_steps_taken} eğitim adımı atıldı.")
            # Son model state'i globalde güncelleyelim ki kaydedilebilsin
            # (Eğer current_train_state bu scope'ta güncellenmiyorsa, global bir değişkene atamak gerekebilir)
            # Ancak JAX'ta state genellikle fonksiyonlardan döndürülür.
            # Bu Colab yapısında, `current_train_state` zaten en dış scope'ta.
            print("Son model durumu (current_train_state) güncellendi.")
        else:
            print("Hiç (yeni) eğitim adımı atılmadı. Lütfen veri ve model yapılandırmasını kontrol edin.")

## Adım 6: İnce Ayarlanmış Modelin Kaydedilmesi ve İndirilmesi (Kavramsal)

Eğitim döngüsü tamamlandıktan sonra (veya istediğiniz bir noktada), ince ayarlanmış modelin ağırlıklarını kaydedebilir ve daha sonra kullanmak üzere indirebilirsiniz.

**YİNE UYARI:** Bu bölüm, `current_train_state`'in (eğitilmiş parametreleri içeren Flax `TrainState`) Adım 4 ve 5'te düzgün bir şekilde elde edildiği ve güncellendiği varsayımına dayanır. Eğer bu state mevcut değilse veya geçerli değilse, kaydetme işlemi **başarısız olacaktır**.

In [ ]:
# @title Modeli Kaydetme ve İndirme Hücresi (Kavramsal Kod)
from flax.training import checkpoints # Flax'in checkpoint yardımcıları
import datetime
import shutil

#@markdown ### Kaydetme Ayarları:
FINETUNED_MODEL_DIR = "fine_tuned_magenta_rt_checkpoint" #@param {type:"string"}
MODEL_FILENAME_PREFIX = "finetuned_mrt_llm" #@param {type:"string"}

if 'current_train_state' not in locals() or current_train_state is None:
  print("HATA: Kaydedilecek model eğitim durumu (current_train_state) bulunamadı veya geçerli değil.")
  print("Lütfen önceki adımların (özellikle Adım 4 ve 5) başarıyla tamamlandığından ve `current_train_state`'in güncellendiğinden emin olun.")
elif not hasattr(current_train_state, 'params') or not hasattr(current_train_state, 'step'):
  print("HATA: `current_train_state` beklenen `params` veya `step` özelliklerine sahip değil. Kaydedilemiyor.")
  print("Bu, Adım 4'teki TrainState oluşturma işleminin başarısız olduğu anlamına gelebilir.")
else:
  try:
    if not os.path.exists(FINETUNED_MODEL_DIR):
      os.makedirs(FINETUNED_MODEL_DIR)
      print(f"'{FINETUNED_MODEL_DIR}' dizini oluşturuldu.")
    
    # Flax TrainState'i kaydetme
    # `target` olarak tüm TrainState nesnesini veriyoruz, bu hem parametreleri hem de optimizatör durumunu içerir.
    save_path = checkpoints.save_checkpoint(
        ckpt_dir=FINETUNED_MODEL_DIR,
        target=current_train_state, # Kaydedilecek nesne (Flax TrainState)
        step=current_train_state.step, # TrainState'in içindeki adım sayısını kullan
        prefix=MODEL_FILENAME_PREFIX + "_",
        overwrite=True # Önceki aynı adımdaki checkpoint'lerin üzerine yaz
    )
    print(f"Model eğitim durumu (checkpoint) başarıyla '{save_path}' olarak kaydedildi.")

    # Kaydedilen checkpoint dizinini sıkıştırıp indirme
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Checkpoint genellikle bir dizin olduğundan (örn: fine_tuned_magenta_rt_checkpoint/finetuned_mrt_llm_100)
    # tüm FINETUNED_MODEL_DIR'i sıkıştırmak daha mantıklı.
    zip_filename_base = f"{FINETUNED_MODEL_DIR}_{timestamp}"
    zip_filepath = f"{zip_filename_base}.zip"
    
    print(f"'{FINETUNED_MODEL_DIR}' dizini '{zip_filepath}' olarak sıkıştırılıyor...")
    shutil.make_archive(zip_filename_base, 'zip', FINETUNED_MODEL_DIR)

    print(f"Sıkıştırılmış model checkpoint'i '{zip_filepath}' indirilmeye hazır.")
    print(f"Lütfen Colab'ın sol tarafındaki dosya gezgininden '{zip_filepath}' dosyasını bulun ve sağ tıklayıp indirin.")
    print("Alternatif olarak, bir sonraki hücreyi kullanarak indirmeyi deneyebilirsiniz (büyük dosyalar için zaman aşımına uğrayabilir).")

  except NotImplementedError as nie:
      print(f"Model kaydedilemedi veya sıkıştırılamadı: {nie}")
  except Exception as e:
    print(f"Model kaydedilirken veya sıkıştırılırken bir hata oluştu: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# @title Modeli İndirme Hücresi (Opsiyonel)
#@markdown Yukarıdaki hücrede sıkıştırılan ZIP dosyasının adını buraya girin.
#@markdown Örneğin: `fine_tuned_magenta_rt_checkpoint_20231027_103045.zip`
FILE_TO_DOWNLOAD_USER_SPECIFIED = "" #@param {type:"string"}

if not FILE_TO_DOWNLOAD_USER_SPECIFIED:
    print("Lütfen indirilecek ZIP dosyasının adını yukarıdaki alana girin.")
elif 'current_train_state' not in locals() or current_train_state is None or not hasattr(current_train_state, 'params'):
    print("Model eğitim durumu düzgün bir şekilde başlatılmadığı veya eğitilmediği için indirme işlemi mantıklı değil.")
elif os.path.exists(FILE_TO_DOWNLOAD_USER_SPECIFIED):
    try:
        print(f"'{FILE_TO_DOWNLOAD_USER_SPECIFIED}' indiriliyor...")
        files.download(FILE_TO_DOWNLOAD_USER_SPECIFIED)
        print("İndirme isteği tarayıcınıza gönderildi. Lütfen tarayıcınızın indirme yöneticisini kontrol edin.")
    except Exception as e:
        print(f"Dosya indirilirken bir hata oluştu: {e}.")
        print("Dosya çok büyükse bu Colab yöntemi zaman aşımına uğrayabilir.")
        print("Lütfen Colab'ın sol tarafındaki dosya gezgininden dosyayı manuel olarak indirmeyi deneyin.")
else:
    print(f"HATA: '{FILE_TO_DOWNLOAD_USER_SPECIFIED}' adlı dosya mevcut dizinde bulunamadı.")
    print("Lütfen dosya adının doğru olduğundan ve bir önceki hücrenin başarıyla çalıştığından emin olun.")
    print(f"Mevcut dizindeki dosyalar ve klasörler: {os.listdir('.')}")

## Adım 7: Sonuç ve Sonraki Adımlar

Tebrikler! Bu not defterindeki adımları (kavramsal olanlar dahil) tamamladınız.

**Özet:**
*   Magenta RT ve bağımlılıklarını kurdunuz.
*   İnce ayar için kendi ses verilerinizi yüklediniz ve ön işlediniz.
*   Model tanımlama, optimizatör ve eğitim döngüsü adımlarının kavramsal bir uygulamasını gördünüz.
*   İnce ayarlanmış modelinizi (kavramsal olarak) kaydetme ve indirme adımlarını incelediniz.

**ÖNEMLİ HATIRLATMA:** Bu not defterindeki Adım 4 (Model Tanımlama) ve Adım 5 (Eğitim Döngüsü), Magenta RT'nin T5X tabanlı LLM'inin karmaşıklığı nedeniyle **büyük ölçüde kavramsal ve yer tutucudur.** Bu adımların tam olarak çalışır hale getirilmesi, T5X model mimarisine derinlemesine hakimiyet ve muhtemelen T5X'in kendi eğitim/ince ayar araçlarının kullanılmasını gerektirir. Bu Colab, bu sürecin genel akışını ve zorluklarını göstermeyi amaçlamaktadır.

**Sonraki Adımlar (Eğer İnce Ayar Başarılı Olsaydı):**
1.  **Modeli Yükleme:** Kaydettiğiniz ince ayarlanmış checkpoint'i kullanarak bir `MagentaRT` nesnesini yeniden başlatmanız gerekir. Bu, `system.MagentaRT` kurucusuna uygun `checkpoint_dir` parametresini vermeyi içerebilir.
2.  **Test Etme:** İnce ayarlanmış modelin, orijinal modele kıyasla veya ince ayar verilerinize daha uygun çıktılar üretip üretmediğini test edin. Farklı istemler (prompts) kullanarak müzik üretin.
3.  **Daha Fazla İyileştirme:** Eğitim parametrelerini (öğrenme hızı, epoch sayısı, batch boyutu), veri ön işleme adımlarını veya ince ayar stratejisini (örneğin, hangi katmanların eğitileceği) değiştirerek sonuçları iyileştirmeyi deneyebilirsiniz.

Bu not defterinin, Magenta RT ile ince ayar sürecine bir giriş yapmanıza yardımcı olduğunu umuyoruz!

## Adım 8: İnce Ayarlanmış Modeli Test Etme (Kavramsal)

Eğer Adım 4, 5 ve 6'daki ince ayar, model kaydetme ve yükleme işlemleri başarıyla tamamlanabilseydi, bu adımda ince ayarlanmış modelinizin performansını test edebilirdiniz.

**Test Etme Adımları (Genel Fikir):**
1.  **İnce Ayarlanmış Modeli Yükleme:** `MagentaRT` sistemini, kaydettiğiniz ince ayarlanmış checkpoint'i gösterecek şekilde başlatmanız gerekir. Bu, `system.MagentaRT` kurucusundaki `checkpoint_dir` argümanını kullanmayı içerebilir. `tag` argümanı, orijinal modelin boyutunu (base veya large) belirtmeye devam edebilirken, `checkpoint_dir` özel ağırlıklarınızı işaret eder.
2.  **Müzik Üretme:** İnce ayar yaparken kullandığınız veri stiline benzer veya o stili çağrıştıran metin istemleri (prompts) kullanarak yeni müzik parçaları üretin.
3.  **Karşılaştırma:**
    *   İnce ayarlanmış modelin çıktılarını, **orijinal (ince ayar yapılmamış) Magenta RT modelinin** aynı istemlerle ürettiği çıktılarla karşılaştırın.
    *   Çıktıların, ince ayar veri kümenizdeki müziğin karakteristik özelliklerini (tempo, armoni, ritim, enstrümantasyon vb.) ne kadar yansıttığını değerlendirin.
    *   Modelin hala genel müzikal tutarlılığını koruyup korumadığını kontrol edin.

**Aşağıdaki kod hücresi, bu test sürecinin nasıl görünebileceğine dair KAVRAMSAL bir örnektir.** Bu hücre, önceki adımların (özellikle `current_train_state` ve çalışan bir eğitim döngüsü) tam olarak işlevsel olduğunu varsayar, ki bu Colab'ın mevcut halinde durum böyle değildir.

In [ ]:
# @title İnce Ayarlanmış Modeli Test Etme (Kavramsal Kod)
import IPython.display as ipd

#@markdown ### Test Parametreleri:
FINETUNED_CHECKPOINT_PATH = "fine_tuned_magenta_rt_checkpoint/" #@param {type:"string"}
#@markdown Yukarıda Adım 6'da modelinizi kaydettiğiniz dizinin yolu.
TEST_PROMPT = "ince ayar verilerimle ilişkili bir stil" #@param {type:"string"}
NUM_TEST_CHUNKS = 3 #@param {type:"integer"}

mrt_finetuned = None
can_test_finetuned_model = False

print("KAVRAMSAL TEST ADIMI")
print("-------------------------")
if 'current_train_state' not in locals() or current_train_state is None or not hasattr(current_train_state, 'params'):
    print("UYARI: Geçerli bir ince ayarlanmış model durumu (`current_train_state`) bulunmadığından bu test çalıştırılamaz.")
    print("Bu, Adım 4 ve 5'teki kavramsal adımların tam olarak uygulanamamasından kaynaklanmaktadır.")
elif not os.path.exists(FINETUNED_CHECKPOINT_PATH) or not os.listdir(FINETUNED_CHECKPOINT_PATH):
    print(f"UYARI: Belirtilen ince ayarlanmış checkpoint yolu '{FINETUNED_CHECKPOINT_PATH}' bulunamadı veya boş.")
    print("Lütfen Adım 6'nın (kavramsal kaydetme) en azından bir dosya oluşturduğundan emin olun.")
else:
    print("İnce ayarlanmış Magenta RT modeli yüklenmeye çalışılıyor (kavramsal)...")
    try:
        # Gerçek bir senaryoda, MagentaRT kurucusunun `checkpoint_dir` parametresini kullanarak
        # T5X modelinin belirli bir checkpoint'ten yüklenmesini sağlamak gerekir.
        # Bu, `magenta_rt.depthformer.model.load_pretrained_model` fonksiyonunun bu özelliği desteklemesini gerektirir.
        # Mevcut `system.MagentaRT` sınıfı, `checkpoint_dir` argümanını doğrudan T5X yükleyicisine geçiriyor olabilir.
        
        mrt_finetuned = system.MagentaRT(
            tag=MODEL_TAG,  # Orijinal modelin temel etiketini koru (base veya large)
            device="tpu:v2-8", 
            skip_cache=True, # Kontrol noktalarını yerel olarak aramaması için
            lazy=False,      # Test için modeli hemen yükle
            checkpoint_dir=FINETUNED_CHECKPOINT_PATH # İnce ayarlanmış checkpoint'in yolu
        )
        print(f"'{FINETUNED_CHECKPOINT_PATH}' yolundan ince ayarlanmış model yüklendi (varsayımsal olarak).")
        can_test_finetuned_model = True
    except Exception as e:
        print(f"İnce ayarlanmış model yüklenirken HATA oluştu: {e}")
        print("Bu, `system.MagentaRT`'nin `checkpoint_dir` ile özel ağırlıkları yükleme yeteneğine bağlıdır.")
        print("Mevcut Magenta RT sürümü bu işlevselliği tam olarak desteklemiyor olabilir.")

if can_test_finetuned_model and mrt_finetuned is not None:
    print(f"\n'${TEST_PROMPT}' istemiyle ince ayarlanmış modelden müzik üretiliyor...")
    
    try:
        style_embedding_ft = mrt_finetuned.embed_style(TEST_PROMPT)
        
        chunks_ft = []
        current_state_ft = None
        for i in range(NUM_TEST_CHUNKS):
            chunk_audio, current_state_ft = mrt_finetuned.generate_chunk(
                state=current_state_ft,
                style=style_embedding_ft,
                seed=i # Her parça için farklı tohum
            )
            chunks_ft.append(chunk_audio)
            print(f"  Parça {i+1}/{NUM_TEST_CHUNKS} üretildi.")
        
        if chunks_ft:
            concatenated_audio_ft = audio.concatenate(
                chunks_ft,
                crossfade_time=mrt_finetuned.config.crossfade_length
            )
            print("Üretilen ses (ince ayarlanmış model):")
            ipd.display(ipd.Audio(concatenated_audio_ft.samples.T, rate=concatenated_audio_ft.sample_rate, normalize=False))
        else:
            print("İnce ayarlanmış modelden hiç parça üretilemedi.")

    except Exception as e:
        print(f"İnce ayarlanmış modelle müzik üretirken HATA oluştu: {e}")
        import traceback
        traceback.print_exc()
        print("Bu, yüklenen checkpoint'in modelle uyumsuz olmasından veya çalışma zamanı hatalarından kaynaklanabilir.")
        
    # Karşılaştırma için orijinal modeli de çalıştırabilirsiniz (eğer hala `MRT` değişkeninde yüklüyse)
    if 'MRT' in locals() and MRT is not None and MRT != mrt_finetuned:
        print(f"\nKarşılaştırma için orijinal modelden ('{TEST_PROMPT}') müzik üretiliyor...")
        try:
            style_embedding_orig = MRT.embed_style(TEST_PROMPT)
            chunks_orig = []
            current_state_orig = None
            for i in range(NUM_TEST_CHUNKS):
                chunk_audio_orig, current_state_orig = MRT.generate_chunk(state=current_state_orig, style=style_embedding_orig, seed=i)
                chunks_orig.append(chunk_audio_orig)
            concatenated_audio_orig = audio.concatenate(chunks_orig, crossfade_time=MRT.config.crossfade_length)
            print("Üretilen ses (orijinal model):")
            ipd.display(ipd.Audio(concatenated_audio_orig.samples.T, rate=concatenated_audio_orig.sample_rate, normalize=False))
        except Exception as e:
            print(f"Orijinal modelle müzik üretirken HATA oluştu: {e}")
else:
    if not can_test_finetuned_model:
        print("\nİnce ayarlanmış model yüklenemediği veya test için uygun olmadığı için test adımı atlandı.")

print("\nKavramsal Test Adımı Tamamlandı.")